# Introduction

This notebook presents the [Initial Data Analysis](#Initial-Data-Analysis) and the [Data Manipulation](#Data-Manipulation) of the New York Times Blog dataset. The first part presents an overview of the data, according to the first course activity. The second part transforms the data in order to improve understanding and results from the analysis. 


A [Summary](#Summary) is presented at the end of the notebook, as well as the [Concluding Remarks](#Concluding-Remarks).

# Initial Data Analysis

This step was performed for the first week of the course. If you are only interested in the Data Manipulation, skip the next subsections.

## Loading necessary libraries

In [1]:
import pandas as pd # Pandas for Data Frame 
import numpy as np # NumPy for vetorial operations

## Loading the dataset

In [2]:
# Reads the CSV file using Pandas
dataset = pd.read_csv("NYTimesBlogTrain.csv")

## Presenting the first 5 rows

This initial view shows the attributes available and their content. As the table shows, there is at least one column with missing values.

In [3]:
# Presents the top 5 rows
dataset.head()

,NewsDesk,SectionName,SubsectionName,Headline,Snippet,Abstract,WordCount,PubDate,Popular,UniqueID
0,Business,Crosswords/Games,NaN,More School Daze,A puzzle from Ethan Cooper that reminds me tha...,A puzzle from Ethan Cooper that reminds me tha...,508,2014-09-01 22:00:09,1,1
1,Culture,Arts,NaN,New 96-Page Murakami Work Coming in December,The Strange Library will arrive just three and...,The Strange Library will arrive just three and...,285,2014-09-01 21:14:07,0,2
2,Business,Business Day,Dealbook,Public Pension Funds Stay Mum on Corporate Expats,Public pension funds have major stakes in Amer...,Public pension funds have major stakes in Amer...,1211,2014-09-01 21:05:36,0,3
3,Business,Business Day,Dealbook,Boot Camp for Bankers,As they struggle to find new business to bolst...,As they struggle to find new business to bolst...,1405,2014-09-01 20:43:34,1,4
4,Science,Health,NaN,Of Little Help to Older Knees,Middle-aged and older patients are unlikely to...,Middle-aged and older patients are unlikely to...,181,2014-09-01 18:58:51,1,5


## Identifying the period in which the news were recorded

By knowing the period, it is possible to estimate, for future steps, which time features could be used. By the fact that there are data corresponding to 3 months, it possible to discard month as a target for analysis, since there is not data for all months.

In [4]:
# Extracts the minimum and maximum date. By the fact that PubDate is organized in format: 
# YEAR-MONTH-DAY HOUR:MINUTE:SECOND, the lexicographic order corresponds to the chronological order.
"From {0} to {1}".format(dataset["PubDate"].min(), dataset["PubDate"].max())

'From 2014-09-01 00:01:32 to 2014-11-30 22:01:45'

## Selecting only columns of interest

For this initial analysis, only 4 variables will be analyzed.

In [5]:
# Creates a new dataframe using only specific columns
sub_dataset = dataset[["NewsDesk", "SectionName", "WordCount", "PubDate", "Popular"]]

## Counting the number of null values per column

Here we observe with detail for missing value on interest columns. As the results show, *NewsDesk* and *SectionName* contain missing values.

In [6]:
# A new data frame is created.For each attribute, the number of null values summed and results in a column
empty_values_summary = pd.DataFrame(
    {'NewsDesk': sub_dataset['NewsDesk'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'SectionName': sub_dataset['SectionName'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'WordCount': sub_dataset['WordCount'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'PubDate': sub_dataset['PubDate'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'Popular': sub_dataset['Popular'].isnull().apply(lambda x: 1 if x else 0).sum()},
      index=['Null Values'])

# Presents the dataframe created
empty_values_summary

,NewsDesk,Popular,PubDate,SectionName,WordCount
Null Values,1846,0,0,2300,0


## Summarizing the information about WordCount

The first variable analyzed is *WordCount*. The code below show statistics about the number of words from all publications. As it is possible to observe, there are publications without word and at least one with 10,912 words. In average, the number of words is 546.

In [7]:
# Select only WordCount and use describe function to extract the main statistics from values.
sub_dataset[['WordCount']].describe()

,WordCount
count,6532.000000
mean,524.434323
std,546.153272
min,0.000000
25%,187.000000
50%,374.000000
75%,723.250000
max,10912.000000


## Identifying the number of publications without words

To better estimate the number of publications without words, a specific count is performed.

In [8]:
# Selects only records with WordCount equals to 0 and count the number
sub_dataset[ sub_dataset['WordCount'] == 0 ]['WordCount'].count()

88

## Presenting the distribuition of NewsDesk values

*NewsDesk* is the category of a publication. The code below shows the percentage of publications for each *NewsDesk* value. The most frequent category is **Business**, while the less frequent is **Sports**.

In [9]:
sub_dataset['NewsDesk'].value_counts(sort=True, normalize=True)

Business    0.236987
TStyle      0.110839
Culture     0.103491
OpEd        0.079761
Foreign     0.057410
Styles      0.045468
Metro       0.030312
Science     0.029700
Travel      0.017759
Magazine    0.004746
National    0.000612
Sports      0.000306
dtype: float64

## Observing the distribuition of NewsDesk values by Popularity

The code above showed, proportionally, the number of publications by *NewsDesk*. Now we analyze, for each *NewsDesk*, the division between popular (indicated by *1*) and impopular (indicated by *0*) entries.

In [10]:
# Groups records by NewsDesk and Popular and counts the number of values
sub_dataset.groupby(["NewsDesk", "Popular"]).size()

NewsDesk  Popular
Business  0          1301
          1           247
Culture   0           626
          1            50
Foreign   0           372
          1             3
Magazine  0            31
Metro     0           181
          1            17
National  0             4
OpEd      0           113
          1           408
Science   0            73
          1           121
Sports    0             2
Styles    0           196
          1           101
TStyle    0           715
          1             9
Travel    0           115
          1             1
dtype: int64

## Observing the distribution of SectionName values by Popularity

In [11]:
# Groups the records by NewsDesk, SectionName and Popular and counts the number of values
sub_dataset.groupby(["NewsDesk", "SectionName", "Popular"]).size()

NewsDesk  SectionName       Popular
Business  Business Day      0          998
                            1           93
          Crosswords/Games  0           19
                            1          103
          Technology        0          280
                            1           50
Culture   Arts              0          625
                            1           50
Foreign   World             0          209
                            1            3
Magazine  Magazine          0           31
Metro     N.Y. / Region     0          181
                            1           17
National  U.S.              0            2
OpEd      Opinion           0          113
                            1          407
Science   Health            0           73
                            1          119
Sports    Sports            0            1
Styles    Health            0            1
          Style             0            2
          U.S.              0           77
                  

## Analyzing SectionNames distribution

In [12]:
sub_dataset['SectionName'].value_counts(sort=True, normalize=True)

Business Day        0.167177
Arts                0.103337
Opinion             0.092927
U.S.                0.077312
Technology          0.050521
World               0.032456
N.Y. / Region       0.030312
Health              0.029700
Multimedia          0.021586
Crosswords/Games    0.018830
Travel              0.017912
Magazine            0.004746
Open                0.000612
Style               0.000306
Sports              0.000153
dtype: float64

## Analyzing Popular distribution

In [13]:
sub_dataset['Popular'].value_counts(sort=True, normalize=True)

0    0.83267
1    0.16733
dtype: float64

# Data Manipulation

In this section some data preparation steps are performed to segment and extract information from dataset attributes.

In order to better present the results a function was created to present absolute and relative frequency values.

In [14]:
def create_frequency_table(dataset, reference_column):
    """Create a data frame with absolute and relative frequency of a reference column

    Arguments:
      * dataset: a data frame
      * reference_column: the reference column from which the frequency will be computed

    Returns:
       * a data frame with relative and absolute frequency of a column

    """
    frequency_table = dataset.groupby([reference_column], as_index=False).size()
    return pd.DataFrame({reference_column: frequency_table.index.values, 
             'Absolute Frequency': frequency_table.values,
             'Relative Frequency': ["{0:.2f}%".format(x/len(dataset)*100) for x in frequency_table.values] },
             columns=[reference_column, 'Absolute Frequency', 'Relative Frequency'])

The first manipulation performed is binning (or segmenting) the *WordCount* variable. 

In [15]:
# Avoid a warning that incorrecly appears when performing the assignment below.
pd.options.mode.chained_assignment = None
# Create WordCountBins with 7 bins
sub_dataset['WordCountBins'] = pd.cut(sub_dataset['WordCount'], bins = [0, 1, 128, 256, 512, 1024, 2048, 1048576], include_lowest=True, right=True)

As result of the binning, the distribution is:

In [16]:
create_frequency_table(sub_dataset, 'WordCountBins')

,WordCountBins,Absolute Frequency,Relative Frequency
0,"[0, 1]",92,1.41%
1,"(1, 128]",1006,15.40%
2,"(128, 256]",1253,19.18%
3,"(256, 512]",1770,27.10%
4,"(512, 1024]",1583,24.23%
5,"(1024, 2048]",705,10.79%
6,"(2048, 1048576]",123,1.88%


As we want to analyze if time properties have influence on the popularity, we will extract some features from *PubDate*.

The features extracted are: *Day*, *Hour* and *Week Day* and a field that indicates if the publication was perfomed in a weekend or not.

In [17]:
# Convert PubDate to Datetime Index
datetime_index = pd.DatetimeIndex(sub_dataset["PubDate"])

# Extract features from DateTime
sub_dataset["PubMonth"] = datetime_index.month
sub_dataset["PubWeekday"] = datetime_index.weekday
sub_dataset["PubDay"] = datetime_index.day
sub_dataset["PubHour"] = datetime_index.hour

# Indicates with True if the Week Day is Saturday (5) or Sunday (6)
sub_dataset["IsWeekend"] = sub_dataset["PubWeekday"].apply(lambda wd: wd in [5, 6] )

The news distribution across months

In [18]:
create_frequency_table(sub_dataset, 'PubMonth')

,PubMonth,Absolute Frequency,Relative Frequency
0,9,2341,35.84%
1,10,2382,36.47%
2,11,1809,27.69%


The news distribution across week days

In [19]:
create_frequency_table(sub_dataset, 'PubWeekday')

,PubWeekday,Absolute Frequency,Relative Frequency
0,0,1224,18.74%
1,1,1190,18.22%
2,2,1224,18.74%
3,3,1228,18.80%
4,4,1164,17.82%
5,5,190,2.91%
6,6,312,4.78%


The news distribution across days

In [20]:
create_frequency_table(sub_dataset, 'PubDay')

,PubDay,Absolute Frequency,Relative Frequency
0,1,150,2.30%
1,2,211,3.23%
2,3,266,4.07%
3,4,204,3.12%
4,5,200,3.06%
5,6,231,3.54%
6,7,218,3.34%
7,8,212,3.25%
8,9,208,3.18%
9,10,295,4.52%


The news distribution across hours

In [21]:
create_frequency_table(sub_dataset, 'PubHour')

,PubHour,Absolute Frequency,Relative Frequency
0,0,125,1.91%
1,1,28,0.43%
2,2,24,0.37%
3,3,60,0.92%
4,4,171,2.62%
5,5,251,3.84%
6,6,220,3.37%
7,7,394,6.03%
8,8,338,5.17%
9,9,316,4.84%


The news distribution across business days and weekends

In [22]:
create_frequency_table(sub_dataset, 'IsWeekend')

,IsWeekend,Absolute Frequency,Relative Frequency
0,False,6030,92.31%
1,True,502,7.69%


Finally, we analyze if any of the new variables contains null values. Based on the fact that *WordCount* and *PubDate* do not contain null values, the columns derived from them also do not contain.

In [23]:
# A new data frame is created.For each attribute, the number of null values summed and results in a column
empty_values_summary = pd.DataFrame(
    {'NewsDesk': sub_dataset['NewsDesk'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'SectionName': sub_dataset['SectionName'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'WordCount': sub_dataset['WordCount'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'Popular': sub_dataset['Popular'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'WordCountBins': sub_dataset['WordCountBins'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'PubMonth': sub_dataset['PubMonth'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'PubWeekday': sub_dataset['PubWeekday'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'PubDay': sub_dataset['PubDay'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'PubHour': sub_dataset['PubHour'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'IsWeekend': sub_dataset['IsWeekend'].isnull().apply(lambda x: 1 if x else 0).sum()},
     index=['Null Values'])

# Presents the dataframe created
empty_values_summary

,IsWeekend,NewsDesk,Popular,PubDay,PubHour,PubMonth,PubWeekday,SectionName,WordCount,WordCountBins
Null Values,0,1846,0,0,0,0,0,2300,0,0


# Summary

The original variable analyzes showed us that:
 * **WordCount**: does not contain null values, but has 88 zero values. The values ranges from 0 to 10912 words. Since it is a numeric attribute, it makes sense to analyze the frequency values by the distribution statistics. The *mean* is 524.43 and the *standard deviation* is 546.15. 
 * **NewsDesk**: there are 12 *NewsDesk* values. The three most common are *Business* (23%), *TStyle* (11%) and *Culture* (10%). From all entries, there are 1846 missing *NewsDesk* values.
 * **SectionName**: there are 14 *SectionName* in the dataset. Some of them a related to more than one *NewsDesk* (e.g. *U.S.* is related to *National* and *Styles*). The three most frequent entries are *Business Day* (16%), *Arts* (10%) and *Opinion* (9%). There are 2300 missing values.
 * **Popularity**: is contains only 2 values: *0* and *1*. Since it is used as a dependent variable on the dataset, it does not contain any missing value. Only 16% of the records are popular.
 
 The analysis of the new attributes indicated the absence of null values. Individually, it is possible to observe that:
 * **WordCountBins**: contains 7 values. The distribution if from 92 to 1770 value per bin.
 * **PubMonth**: contains 3 month, each of which contains about 2000 records.
 * **PubWeekDay**: naturally contains 7 values. Business days contains about 1000 records, while each weekend day contains less than 400.
 * **PubDay**: contains 31 days. Except for the 31th day, which contains 95 days, the days contains about 200 records.
 * **PubHour**: contains 24 hours. The period between 7 and 18 hours contains the higher number of records.
 * **IsWeekend**: as indicated by the PubWeekDay distrubution, business days contains the majority of records (6030), while weekends contains 502.

# Concluding Remarks

In this activity the New York Time blog dataset was analyzed and some attributes were manipulated to segment and extract features. 

The original attributes analyzed were:
 * WordCount
 * NewsDesk
 * SectionName 
 * Popularity
 * PubDate
 
 The new attributes created were:
 
 * WordCountBins bin.
 * PubMonth
 * PubWeekDay
 * PubDay
 * PubHour
 * IsWeekend
 
It is also possible to analyze the textual content, such as snippet and abstract, to observe the content influence on the publication popularity. However, this step will be left for remaining course modules to avoid creating a overly extensive notebook.